# Financial Econometrics II: Final Project

*By Basri Satiroglu, Daniel Deutsch and José Lucas Barretto*

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Matplotlib styles
plt.style.use('ggplot')
plt.rcParams.update({
    'figure.figsize': (15, 7),
    'axes.prop_cycle': plt.cycler(color=['#4C72B0', '#C44E52', '#55A868', '#8172B2', '#CCB974', '#64B5CD']),
    'axes.facecolor': '#EAEAF2'
})

## Section 1

### Data Loading

In [14]:
# read raw dataset
df_raw = pd.read_stata('Diane-data.dta')

# rename columns
df_raw.rename(
    columns = {
        'siren': 'siren_float',
        'sirenID': 'siren',
        'companyname': 'company',
        'regioncode': 'region_code',
        'départementcode': 'department_code',
        'cashNbank': 'cash',
        'totalDebt': 'totaldebt'
    },
    inplace=True
)

# select desired columns from data
columns = [
    'siren', 'year', 'company', 'totalassets', 'marketable_sec', 'cash', 
    'totaldebt', 'inventory', 'creditors', 'tradedebts', 'taxes', 'otherdebts', 
    'netincome'
]

# filter desired department and columns
df = df_raw.loc[df_raw['department_code'] == 33, columns]

# drop na values
df.dropna(inplace=True)

# set siren and date as indexes
df = df.sort_values(['siren', 'year']).set_index(['siren', 'year'], verify_integrity=True)

### Data Transformations

#### Cash + Marketable Securities Ratio

In [15]:
# create filters for numerator and denominator
mask_num = (df['cash'] + df['marketable_sec']) >= 0
mask_den = df['totalassets'] >= 1

# compute ratio
df['cash_mktsec_ratio'] = ((df['cash'] + df['marketable_sec'])[mask_num]) / (df['totalassets'][mask_den])

#### Other Ratios

In [16]:
# create filter for ratios
mask = (df['cash'] >= 0) & (df['marketable_sec'] >= 0) & (df['totaldebt'] >= 0)

# calculate working capital
df['nwc'] = (df['inventory'] + df['creditors'] - df['tradedebts'] - df['taxes'] - df['otherdebts'])/df['totalassets']

# calculate other ratios
df['netincome_ratio'] = (df['netincome']/df['totalassets'])[mask]
df['totaldebt_ratio'] = (df['totaldebt']/df['totalassets'])[mask]
df['inventory_ratio'] = df['inventory']/df['totalassets']  
df['creditors_ratio'] = df['creditors']/df['totalassets']  
df['tradedebts_ratio'] = df['tradedebts']/df['totalassets']  
df['taxes_ratio'] = df['taxes']/df['totalassets']  
df['otherdebts_ratio'] = df['otherdebts']/df['totalassets']  

#### Between and Within Transformations

In [24]:
between = df.groupby(level=0).mean()
within = df.subtract(df.groupby(level=0).mean(), level=0)

## Section 2